In [0]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import tensorflow as tf; print(tf.__version__)

1.15.2


In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 4.2MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = '/content/drive/My Drive/security_test_models'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: /content/drive/My Drive/security_test_models *****


#Data

In [0]:
from tensorflow import keras
import os
import re

# Load all files from a directory in a DataFrame.
train_text=pd.read_csv('/content/drive/My Drive/train_raw_sentences.csv',dtype='str')
test_text=pd.read_csv('/content/drive/My Drive/test_raw_sentences.csv',dtype='str')
#train.to_csv('/content/drive/My Drive/train_data.csv')
#test.to_csv('/content/drive/My Drive/train_data.csv')

In [0]:
train_text

,0
0,"The currently used Rails version, in the stabl..."
1,"'This is a useful security improvement, that I..."
2,We just had an admin accidentally push to a br...
3,'It's possible to check publicly if a private ...
4,I agree that people are using self-signed cert...
...,...
103867,Cross-site scripting (XSS) vulnerability in in...
103868,SQL injection vulnerability in dispatch.php in...
103869,Cisco ASR 1000 devices with software before 3....
103870,Multiple cross-site scripting (XSS) vulnerabil...


In [0]:
train_text.head()

,0
0,"The currently used Rails version, in the stabl..."
1,"'This is a useful security improvement, that I..."
2,We just had an admin accidentally push to a br...
3,'It's possible to check publicly if a private ...
4,I agree that people are using self-signed cert...


In [0]:
test_text.head()

,0
0,"Vulnerability in the Java SE, Java SE Embedded..."
1,Remove duplicate content issue in doc template...
2,There's a bug in the GitHub importer library t...
3,Add Python 3.6 classifier to the Python packag...
4,The save_submission function in mod/assign/ext...


In [0]:
train_labels=pd.read_csv('/content/drive/My Drive/train_labels.csv')
test_labels=pd.read_csv('/content/drive/My Drive/test_labels.csv')

In [0]:
label_list = [0,1]

In [0]:
print(train_labels.shape)
print(train_text.shape)

(103872, 1)
(103872, 1)


In [0]:
print(test_labels.shape)
print(test_text.shape)

(11543, 1)
(11543, 1)


In [0]:
train_text.columns=['Text']
train_text.head()

,Text
0,"The currently used Rails version, in the stabl..."
1,"'This is a useful security improvement, that I..."
2,We just had an admin accidentally push to a br...
3,'It's possible to check publicly if a private ...
4,I agree that people are using self-signed cert...


In [0]:
test_text.columns=['Text']
test_text.head()

,Text
0,"Vulnerability in the Java SE, Java SE Embedded..."
1,Remove duplicate content issue in doc template...
2,There's a bug in the GitHub importer library t...
3,Add Python 3.6 classifier to the Python packag...
4,The save_submission function in mod/assign/ext...


In [0]:
train_labels.columns=['Label']
test_labels.columns=['Label']

In [0]:
train_labels['Label'].value_counts()

1    52150
0    51722
Name: Label, dtype: int64

In [0]:
test_labels['Label'].value_counts()

1    5793
0    5750
Name: Label, dtype: int64

In [0]:
train=pd.concat([train_text,train_labels],axis=1)
test=pd.concat([test_text,test_labels],axis=1)

In [0]:
train.head()

,Text,Label
0,"The currently used Rails version, in the stabl...",1
1,"'This is a useful security improvement, that I...",1
2,We just had an admin accidentally push to a br...,1
3,'It's possible to check publicly if a private ...,1
4,I agree that people are using self-signed cert...,1


In [0]:
train_label_diff=train_labels['Label'].subtract(train['Label'], fill_value=3)

In [0]:
train_label_diff.value_counts()

0    103872
Name: Label, dtype: int64

In [0]:
test_label_diff=test_labels['Label'].subtract(test['Label'], fill_value=3)
test_label_diff.value_counts()

0    11543
Name: Label, dtype: int64

In [0]:
def create_examples(lines, set_type, labels=None):
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = str(line)
            label = label
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            #print(examples)
    else:
        for line in lines:
            text_a = str(line)
            label = 0
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

train_InputExamples=create_examples(train_text, 'train',labels=train_labels)
test_InputExamples=create_examples(test_text,'test',labels=test_labels)

In [0]:
type(train_text['Text'])

pandas.core.series.Series

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = str(x['Text']), 
                                                                   text_b = None, 
                                                                   label = int(x['Label'])), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = str(x['Text']), 
                                                                   text_b = None,
                                                                   label = int(x['Label'])), axis = 1)


In [0]:
train_InputExamples

0         <bert.run_classifier.InputExample object at 0x...
1         <bert.run_classifier.InputExample object at 0x...
2         <bert.run_classifier.InputExample object at 0x...
3         <bert.run_classifier.InputExample object at 0x...
4         <bert.run_classifier.InputExample object at 0x...
                                ...                        
103867    <bert.run_classifier.InputExample object at 0x...
103868    <bert.run_classifier.InputExample object at 0x...
103869    <bert.run_classifier.InputExample object at 0x...
103870    <bert.run_classifier.InputExample object at 0x...
103871    <bert.run_classifier.InputExample object at 0x...
Length: 103872, dtype: object

In [0]:
test_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
                               ...                        
11538    <bert.run_classifier.InputExample object at 0x...
11539    <bert.run_classifier.InputExample object at 0x...
11540    <bert.run_classifier.InputExample object at 0x...
11541    <bert.run_classifier.InputExample object at 0x...
11542    <bert.run_classifier.InputExample object at 0x...
Length: 11543, dtype: object

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
# BERT_MODEL_HUB="https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)

    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 103872


INFO:tensorflow:Writing example 0 of 103872


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:Writing example 10000 of 11543


INFO:tensorflow:Writing example 10000 of 11543


#Creating a model

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
OUTPUT_DIR='/content/drive/My Drive/security_test_output'

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/security_test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff3ec0726d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/security_test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff3ec0726d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


Training took time  0:00:00.017784


Now let's use our test data to see how well our model did:

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 9738: auc = 0.5, eval_accuracy = 0.5018626, f1_score = 0.66832024, false_negatives = 0.0, false_positives = 5750.0, global_step = 9738, loss = 4.6356206, precision = 0.5018626, recall = 1.0, true_negatives = 0.0, true_positives = 5793.0


INFO:tensorflow:Saving dict for global step 9738: auc = 0.5, eval_accuracy = 0.5018626, f1_score = 0.66832024, false_negatives = 0.0, false_positives = 5750.0, global_step = 9738, loss = 4.6356206, precision = 0.5018626, recall = 1.0, true_negatives = 0.0, true_positives = 5793.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9738: /content/drive/My Drive/security_test_output/model.ckpt-9738


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9738: /content/drive/My Drive/security_test_output/model.ckpt-9738


{'auc': 0.5,
 'eval_accuracy': 0.5018626,
 'f1_score': 0.66832024,
 'false_negatives': 0.0,
 'false_positives': 5750.0,
 'global_step': 9738,
 'loss': 4.6356206,
 'precision': 0.5018626,
 'recall': 1.0,
 'true_negatives': 0.0,
 'true_positives': 5793.0}

In [0]:
predict_input_fn = run_classifier.input_fn_builder(\
    features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)

preds=[]
for prediction in predictions:
  preds.append(np.argmax(prediction['probabilities']))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/security_test_output/model.ckpt-9738


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
from sklearn.metrics import accuracy_score,classification_report
print("Accuracy of BERT is:",accuracy_score(test['Label'],preds))

Accuracy of BERT is: 0.5018626007103872


In [0]:
print(classification_report(test['Label'],preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5750
           1       0.50      1.00      0.67      5793

    accuracy                           0.50     11543
   macro avg       0.25      0.50      0.33     11543
weighted avg       0.25      0.50      0.34     11543



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
